In [1]:
import collections
import functools
import pathlib
import time

import pandas as pd
from google.api_core.exceptions import ResourceExhausted
from tqdm.notebook import tqdm
from vertexai.language_models import TextGenerationModel

In [2]:
DATA_DIR = pathlib.Path('../data')

# 3. 医療論文の判定
医療に関連する内容の論文を判定します。  
当初はキーワードマッチングで行おうと思ったのですが、キーワードをいくら追加しても網羅しきれない状況が発生したため、LLM の力を借りることにしました。  
論文のタイトルからその論文が医療関連のものかどうかを判定します。  

In [3]:
DATA_PATH = DATA_DIR / 'papers.csv'
DATA_TITLE_PATH = DATA_DIR / 'papers_title.csv'

SAVE_PATH = DATA_DIR / 'papers_med.csv'

## 3-1. データの準備
必要なのはタイトルのみなので、タイトルと推論結果を格納するデータフレームを作成しておきます。  
また処理に時間がかかるので、途中経過を読み込めるようにしておきます。  

In [4]:
TITLE_COL = 'title'
RELATED_TO_MEDICINE_COL = 'related_to_medicine'
EMPTY = '<EMPTY>'

if not DATA_TITLE_PATH.exists():
    df = pd.read_csv(DATA_PATH)
    df_title = df[[TITLE_COL]].drop_duplicates().copy()
    df_title[RELATED_TO_MEDICINE_COL] = EMPTY
else:
    df_title = pd.read_csv(DATA_TITLE_PATH)

print(f'{len(df_title):,}')
print(f'{len(df_title[df_title[RELATED_TO_MEDICINE_COL] == EMPTY]):,} / {len(df_title):,}')
df_title.head(2)

191,136
0 / 191,136


,title,related_to_medicine
0,PatchTrack: Multiple Object Tracking Using Fra...,No
1,Automated Fake News Detection using cross-chec...,No


In [5]:
PROMPT_TEMPLATE = """
Below is the title of a paper.
Is this paper related to medical field?
Answer with “Yes” or “No”.

title: {title}
answer:
""".strip()

## 3-2. 費用の推定
おおよそどのくらいの費用がかかるか事前に算出しておきます。

In [6]:
# https://cloud.google.com/vertex-ai/pricing
PRICING = {'input': 0.00025 / 1000, 'output': 0.0005 / 1000}


def count_characters(text: str) -> int:
    """PaLMではcharacter単位で課金される
    https://medium.com/@van.evanfebrianto/a-deep-dive-into-monitoring-character-consumption-in-langchain-for-vertexai-ensuring-business-d4b6363802a5
    """
    text_without_spaces = ''.join([char for char in text if not char.isspace()])
    return len(text_without_spaces.encode('utf-8'))


def calculate_cost(prompts: list[str], outputs: list[str]) -> float:
    """費用を計算する"""
    cost = 0
    for prompt, output in zip(prompts, outputs):
        input_cost = count_characters(prompt) * PRICING['input']
        output_cost = count_characters(output) * PRICING['output']
        cost += input_cost + output_cost
    return cost

In [7]:
# 出力は仮置き
cost = calculate_cost(
    prompts=[PROMPT_TEMPLATE.format(title=title) for title in df_title.title],
    outputs=['Yes'] * len(df_title)
)
print(f'${cost:.2f}')

$8.36


## 3-3. 推論
$10未満とのことなので、安心して推論を進めていきます。   

In [8]:
PARAMETERS = {
    'max_output_tokens': 8,
    'temperature': 0
}

model_ = TextGenerationModel.from_pretrained('text-bison@001')


def resource_exhausted_handler(func):
    @functools.wraps(func)
    def _wrapper(*args, **kwargs):
        try:
            result = func(*args, **kwargs)
            return True, result
        except ResourceExhausted:
            time.sleep(30)
            return False, None
    return _wrapper


@resource_exhausted_handler
def predict(prompt: str, parameters: dict[str, float] = PARAMETERS) -> str:
    response = model_.predict(prompt, **parameters)
    return response.text

In [9]:
print(f'Remaining: {len(df_title[df_title[RELATED_TO_MEDICINE_COL] == EMPTY]):,} / {len(df_title):,}')

try:
    for i in tqdm(range(len(df_title)), total=len(df_title)):
        if df_title.at[i, RELATED_TO_MEDICINE_COL] == EMPTY:
            prompt = PROMPT_TEMPLATE.format(title=df_title.at[i, TITLE_COL])

            # ResourceExhausted対策
            success = False
            while not success:
                success, result = predict(prompt)

            df_title.at[i, RELATED_TO_MEDICINE_COL] = result
# とにかく処理が止まったら保存されるようにする
except BaseException as e:
    print(f'{e.__class__.__name__}: {e}')

df_title.to_csv(DATA_TITLE_PATH, header=True, index=False)

Remaining: 0 / 191,136


  0%|          | 0/191136 [00:00<?, ?it/s]

## 3-4. 推論結果の確認
出力が`Yes`と`No`になっているかを確認しておきます。  

In [10]:
counter = collections.Counter(df_title[RELATED_TO_MEDICINE_COL])
for pred, count in sorted(counter.items()):
    print(f'"{pred}": {count:,}')

"No": 175,231
"Yes": 15,905


## 3-5. 結果を元データに反映
推論結果を分析元のデータにくっつけます。  

In [11]:
df = pd.read_csv(DATA_PATH)
df_title = pd.read_csv(DATA_TITLE_PATH)

df = df.merge(df_title, how='left', on=TITLE_COL)
print(sorted(set(df[RELATED_TO_MEDICINE_COL])))

['No', 'Yes']


In [12]:
df.to_csv(SAVE_PATH, header=True, index=False)